# AGI Brain - Quick Chat

**Load your saved brain and start chatting immediately!**

---

**First time?** Use the [full notebook](https://colab.research.google.com/github/jeebus87/agi-brain/blob/master/notebooks/agi_language_learning.ipynb) to train your brain first.

**Runtime:** Go to `Runtime > Change runtime type > T4 GPU`

In [ ]:
# Setup (run once)
!pip install tensorflow numpy matplotlib scipy gradio yt-dlp librosa --quiet
!rm -rf agi-brain && git clone https://github.com/jeebus87/agi-brain.git
%cd agi-brain

import sys
sys.path.insert(0, '.')
print("Setup complete!")

In [ ]:
# Mount Google Drive and load your brain
from google.colab import drive
drive.mount('/content/drive')

from src.interface.brain_persistence import BrainPersistence
from src.language.semantic_memory import SemanticMemory, SemanticConfig
from src.language.speech_generator import SpeechGenerator
from src.language.phoneme_learner import PhonemeLearner, PhonemeConfig

BRAIN_NAME = "my_agi_brain"  # Change if you used a different name

# Initialize components
sem_config = SemanticConfig(n_hard_locations=5000, address_size=500, data_size=500)
memory = SemanticMemory(sem_config)
generator = SpeechGenerator()
phoneme_config = PhonemeConfig(n_input=800, n_phoneme_neurons=500, n_phonemes=20)
learner = PhonemeLearner(phoneme_config)

# Create brain
class Brain:
    def __init__(self):
        self.semantic_memory = memory
        self.speech_generator = generator
        self.phoneme_learner = learner
        self.conversation_history = []
    
    def process(self, text):
        words = text.lower().split()
        self.semantic_memory.learn_from_sentence(words)
        
        all_activations = {}
        for word in words:
            if self.semantic_memory.lookup(word):
                activations = self.semantic_memory.spread_activation(word, depth=2)
                for w, a in activations.items():
                    all_activations[w] = all_activations.get(w, 0) + a
        
        if all_activations:
            top_words = sorted(all_activations.items(), key=lambda x: -x[1])[:3]
            associated = [w for w, a in top_words if w not in words]
            if associated:
                response = f"I think of {', '.join(associated)}"
            else:
                response = "Tell me more"
        else:
            response = f"Learning about {words[0] if words else 'nothing'}"
        
        self.conversation_history.append((text, response))
        return response
    
    def speak(self, text):
        return self.speech_generator.speak(text)

brain = Brain()
persistence = BrainPersistence()

# Load saved brain
if persistence.load_brain(brain, BRAIN_NAME):
    print(f"\nBrain loaded! Vocabulary: {brain.semantic_memory.get_vocabulary_size()} words")
else:
    print("\nNo saved brain found. Use the full notebook first to train your brain.")

In [ ]:
# Launch Chat Interface
import gradio as gr
from IPython.display import Audio, display

def chat(message, history):
    response = brain.process(message)
    return response

def save_brain():
    persistence.save_brain(brain, BRAIN_NAME)
    return f"Saved! Vocabulary: {brain.semantic_memory.get_vocabulary_size()} words"

def get_status():
    return f"""Vocabulary: {brain.semantic_memory.get_vocabulary_size()} words
Concepts: {len(brain.semantic_memory.concepts)}
Conversations: {len(brain.conversation_history)} turns"""

with gr.Blocks(title="AGI Brain Chat") as demo:
    gr.Markdown("# AGI Brain Chat\n\nYour brain learns from every conversation!")
    
    chatbot = gr.ChatInterface(
        chat,
        examples=["hello", "tell me about cats", "what can fly", "dogs are friendly"],
    )
    
    with gr.Row():
        save_btn = gr.Button("Save Brain", variant="primary")
        status_btn = gr.Button("Status")
    
    output = gr.Textbox(label="Info")
    save_btn.click(save_brain, outputs=output)
    status_btn.click(get_status, outputs=output)

demo.launch(share=True)

---

**Remember to click "Save Brain" before closing!**